# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
path = "../output_data/cities.csv"
cities_df = pd.read_csv(path)
cities_df = cities_df[['City','Country','Date', 'Lat', 'Lng', 'Max Temp (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)']]

cities_df.head()


,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Ancud,CL,1599271536,-41.87,-73.82,39.99,91,100,5.75
1,Bluff,NZ,1599271755,-46.60,168.33,52.93,68,81,10.31
2,Castro,BR,1599271755,-24.79,-50.01,61.83,79,0,6.11
3,Punta Arenas,CL,1599271756,-53.15,-70.92,37.40,86,100,5.82
4,Vanimo,PG,1599271756,-2.67,141.30,81.23,77,100,3.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_df[["Lat", "Lng"]]
weights = cities_df["Humidity (%)"]

In [12]:
figure_layout = {
    'width': "100%",
    'height': '400px',
    'padding': '1px',
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False,max_intensity = max(weights),point_radius = 3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

#DATAFRAME with the given conditions
cities2_df = cities_df.loc[(cities_df["Max Temp (F)"] >= 70) & (cities_df["Max Temp (F)"] <= 80) &
                            (cities_df["Wind Speed (mph)"] <= 10) & (cities_df["Cloudiness (%)"] == 0)].dropna()

cities2_df

,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
67,Cayenne,GF,1599271779,4.93,-52.33,78.80,88,0,2.24
134,Jacareacanga,BR,1599271804,-6.22,-57.75,77.95,54,0,0.67
136,San Quintín,MX,1599271572,30.48,-115.95,74.01,73,0,9.26
216,Vostok,RU,1599271570,46.49,135.88,72.86,47,0,5.17
225,Waddān,LY,1599271839,29.16,16.14,73.74,70,0,8.37
229,Gornyye Klyuchi,RU,1599271841,45.25,133.50,75.65,57,0,7.63
236,Salé,MA,1599271843,34.04,-6.82,70.00,100,0,8.93
243,Abu Samrah,SY,1599271845,35.30,37.18,77.25,28,0,4.52
271,Guia Lopes da Laguna,BR,1599271856,-21.46,-56.11,75.42,39,0,3.31
275,Flinders,AU,1599271857,-34.58,150.86,70.00,40,0,7.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#Store into variable named hotel_df
hotel_df = cities2_df[['City','Country','Lat', 'Lng']].copy()
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
67,Cayenne,GF,4.93,-52.33,
134,Jacareacanga,BR,-6.22,-57.75,
136,San Quintín,MX,30.48,-115.95,
216,Vostok,RU,46.49,135.88,
225,Waddān,LY,29.16,16.14,
229,Gornyye Klyuchi,RU,45.25,133.50,
236,Salé,MA,34.04,-6.82,
243,Abu Samrah,SY,35.30,37.18,
271,Guia Lopes da Laguna,BR,-21.46,-56.11,
275,Flinders,AU,-34.58,150.86,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

#for loop through eaxch city
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    #Requests
    hotel_data = requests.get(base_url, params).json()
    #hotel_data 
    
    #Exception handling to save the first hotel name
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"{row['City']} City and the nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")

Cayenne City and the nearest hotel is Royal Amazonia
-------------------------------------------------------------------------
Jacareacanga City and the nearest hotel is HOTEL VIANA
-------------------------------------------------------------------------
San Quintín City and the nearest hotel is Hotel Jardines Baja
-------------------------------------------------------------------------
Could not find information 'Vostok'... skipping.
-------------------------------------------------------------------------
Could not find information 'Waddān'... skipping.
-------------------------------------------------------------------------
Gornyye Klyuchi City and the nearest hotel is Empire
-------------------------------------------------------------------------
Salé City and the nearest hotel is Riad Rabat l'Alcazar
-------------------------------------------------------------------------
Could not find information 'Abu Samrah'... skipping.
----------------------------------------------------

In [16]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
67,Cayenne,GF,4.93,-52.33,Royal Amazonia
134,Jacareacanga,BR,-6.22,-57.75,HOTEL VIANA
136,San Quintín,MX,30.48,-115.95,Hotel Jardines Baja
216,Vostok,RU,46.49,135.88,
225,Waddān,LY,29.16,16.14,
229,Gornyye Klyuchi,RU,45.25,133.50,Empire
236,Salé,MA,34.04,-6.82,Riad Rabat l'Alcazar
243,Abu Samrah,SY,35.30,37.18,
271,Guia Lopes da Laguna,BR,-21.46,-56.11,Pousada Zamora
275,Flinders,AU,-34.58,150.86,Eternity 141


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))